# Import

In [6]:
!pip install mlflow
!pip install torch==1.8.1
!pip install pytorch-lightning
!pip install transformers
!pip install cloudpathlib

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 47 kB 5.7 MB/s  eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [2]:
import sys
sys.path.append('../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from ast import literal_eval
from collections import Counter
from typing import Any, Dict, Optional

In [3]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger


import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

import transformers
from transformers import (
    AdamW,
    AutoConfig,
    AutoModel,
    AutoModelForSequenceClassification,
    T5ForConditionalGeneration,
    T5Tokenizer,
    AutoTokenizer,
)
from transformers.optimization import (
    Adafactor,
    get_linear_schedule_with_warmup,
)

In [7]:
from deep.constants import *
from deep.utils import *

In [8]:
%load_ext autoreload
%autoreload 2

## Data

In [9]:
logged_model = 's3://deep-mlflow-artifact/2/9f216acf38d54ff6b185441a0f80e8b7/artifacts/model'

## Sagemaker Prep

In [10]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
prefix = "pl/example"

## Inference

In [11]:
from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(
    entry_point="inference.py",
    source_dir=str(SCRIPTS_PATH / 'examples/inference-sector-pl'),
    role=role,
    model_data=logged_model,
    framework_version="1.8.1",
    py_version="py3",
)

In [10]:
instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge',
    'ml.c4.xlarge'
]

In [13]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

# set local_mode to False if you want to deploy on a remote
# SageMaker instance

local_mode = False

if local_mode:
    instance_type = "local"
else:
    instance_type = "ml.p2.xlarge"

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

ClientError: An error occurred (404) when calling the HeadObject operation: Not Found

In [ ]:
from mlflow.pytorch import pickle_module

In [ ]:
pickle_module

In [ ]:
import sys
sys.path.append('../../scripts/examples/sector-pl/')
import model

In [ ]:
torch.load('/Users/stefano/Downloads/model.pth', pickle_module=pickle_module)

In [10]:
import mlflow
import torch
logged_model = 's3://deep-mlflow-artifact/2/9f216acf38d54ff6b185441a0f80e8b7/artifacts/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pytorch.load_model(logged_model, map_location=torch.device('cpu'))

2021/07/06 11:41:43 WARNING mlflow.pytorch: Stored model version '1.8.1' does not match installed PyTorch version '1.8.1+cu102'


In [17]:
from torch.utils.data import Dataset 
import pandas as pd

class SectorsDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.excerpt_text = dataframe["excerpt"].tolist() if dataframe is not None else None
        self.max_len = max_len

    def encode_example(self, excerpt_text: str, index=None, as_batch: bool = False):
        inputs = self.tokenizer(
            excerpt_text,
            None,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=True,
        )
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]

        encoded = {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
        }
        if as_batch:
            return {
                "ids": encoded["ids"].unsqueeze(0),
                "mask": encoded["mask"].unsqueeze(0),
                "token_type_ids": encoded["ids"].unsqueeze(0),
            }
        return encoded

    def __len__(self):
        return len(self.excerpt_text)

    def __getitem__(self, index):
        excerpt_text = str(self.excerpt_text[index])
        return self.encode_example(excerpt_text, index)

In [34]:
model_name = 'sentence-transformers/paraphrase-mpnet-base-v2'
data = pd.DataFrame({
    'excerpt': ['hello, how', 'how, hello']
})
tokenizer = AutoTokenizer.from_pretrained(model_name)
dataset = SectorsDataset(data, tokenizer, 200)
loaded_model.cpu()


In [36]:
for batch in dataloader:
    preds = loaded_model.forward(batch)

In [44]:
a = [preds, preds]

In [50]:
list(torch.cat(a).argmax(1).numpy())

[6, 0, 6, 0]

In [ ]:
p

In [42]:
preds.shape

torch.Size([2, 11])

In [40]:
preds.argmax(1)

tensor([6, 0])

In [ ]:
loaded_model.predict(pd.DataFrame({'data': []}))